# Hands-on Lab --- Calculating and Analyzing CLV

## Importing standard packages:

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 2)

## Reading in the original data, and joining with the churn and margin predicted values:

In [2]:
combined_df = pd.read_csv(r'C:\Users\dangt\Desktop\Trinh Phan\OSU Lesson\Fall 2020\Customer Life Time Value\Group Project\combined_data.csv')
churn_df = pd.read_csv(r'C:\Users\dangt\Desktop\Trinh Phan\OSU Lesson\Fall 2020\Customer Life Time Value\Group Project\churn_scores.csv')
margin_df = pd.read_csv(r'C:\Users\dangt\Desktop\Trinh Phan\OSU Lesson\Fall 2020\Customer Life Time Value\Group Project\margin_scores.csv')

combined_df = pd.merge(combined_df, churn_df, on='acct_ID', how='left')
combined_df = pd.merge(combined_df, margin_df, on='acct_ID', how='left')

# it should be noted that Python is case sensitive for variable names.  acct_id above would fail.

combined_df.shape

c:\users\dangt\appdata\local\programs\python\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


MemoryError: Unable to allocate 190. MiB for an array with shape (80, 311163) and data type int64

## Since our margin model had transformed the target variable, we need to untransform the predicted value,
##     using the inverse of our transformation:

In [3]:
combined_df['P_margin_Y_pos_1']=10**combined_df['P_log_margin_Y_pos_1']-50000

# Plotting predicted untransformed values vs actual untransformed values:

plt.figure(figsize=(15,15))
plt.scatter(combined_df['P_margin_Y_pos_1'], combined_df['margin_Y_pos_1'])
plt.xlabel("margin")
plt.ylabel("predicted margin")
plt.title("actual v predicted")
# plt.xticks([])
# plt.yticks([])
print(" ") 
plt.show()

KeyError: 'P_log_margin_Y_pos_1'

# It should be noted:  we are scoring our original data, used for modeling, only for the purposes of analysis.
##  In a "real world" situation, we would most likely be scoring a full population for which we did not actually
##  know the future churn or the future margin.

## We are using a simplified CLV formula, in which we:
###    *  are predicting only annual numbers,
###    *  are assuming that next year's annual prediction are the same values for the next 3 years,
###    *  that our future value is adequately estimated using only 3 future years.

In [4]:
annual_discount_rate = 0.10
combined_df['CFV'] = (combined_df['P_margin_Y_pos_1']*((1-combined_df['P_churn_year_plus1_ind'])**1))/((1+annual_discount_rate)**1) + (combined_df['P_margin_Y_pos_1']*((1-combined_df['P_churn_year_plus1_ind'])**2))/((1+annual_discount_rate)**2) + (combined_df['P_margin_Y_pos_1']*((1-combined_df['P_churn_year_plus1_ind'])**3))/((1+annual_discount_rate)**3)
plothist = combined_df['CFV'].plot.hist(title="Distribution of Customer Future Value",bins=20, range=(-1000,8000),edgecolor='black')
plt.show()
combined_df.describe()

KeyError: 'P_margin_Y_pos_1'

## Establishing the quadrants for high/low churn and margin

In [5]:
print("median predicted churn: ",  np.median(combined_df['P_churn_year_plus1_ind']))
print("median predicted margin: %d" % (np.median(combined_df['P_margin_Y_pos_1'])))

# initializing to values which should not occur after logic below.
combined_df['retention_grp'] = '0X'
combined_df['margin_grp'] = '0X'
combined_df['quadrant_retention_margin'] = 'error_error'

combined_df.loc[combined_df['P_churn_year_plus1_ind']<np.median(combined_df['P_churn_year_plus1_ind']),'retention_grp']="2H"
combined_df.loc[combined_df['P_churn_year_plus1_ind']>=np.median(combined_df['P_churn_year_plus1_ind']),'retention_grp']="1L"

combined_df.loc[(combined_df['P_margin_Y_pos_1']<np.median(combined_df['P_margin_Y_pos_1'])),'margin_grp']="1L"
combined_df.loc[(combined_df['P_margin_Y_pos_1']>=np.median(combined_df['P_margin_Y_pos_1'])),'margin_grp']="2H"

combined_df.loc[(combined_df['retention_grp'] == '1L') & (combined_df['margin_grp'] == '1L'),'quadrant_retention_margin'] = "1L_1L"
combined_df.loc[(combined_df['retention_grp'] == '1L') & (combined_df['margin_grp'] == '2H'),'quadrant_retention_margin'] = "1L_2H"
combined_df.loc[(combined_df['retention_grp'] == '2H') & (combined_df['margin_grp'] == '1L'),'quadrant_retention_margin'] = "2H_1L"
combined_df.loc[(combined_df['retention_grp'] == '2H') & (combined_df['margin_grp'] == '2H'),'quadrant_retention_margin'] = "2H_2H"

combined_df.head(10)

median predicted churn:  0.09414545715292694
median predicted margin: 196


,ST,Zip,DMA,Area_Cd,emailable,solicit_ph_ind,solicit_email_ind,fixd_incm_flg,scheduled_purchase_flg,months_since_last_trans,months_since_first_trans,purch_avg_interval,mail_cnt_36mo,call_in_mo_36mo,call_out_mo_36mo,tier,tier_prev,acct_ID,activation_date,event_date,churn_year_plus1_ind,log_order_cnt_36mo,log_purch_amt_36mo,log_purch_amt_life,log_purch_avg_36mo,...,demog_LOR_imp_06 to 10 years,demog_LOR_imp_11 to 19 years,demog_LOR_imp_20+ years,demog_ownrent_imp_Definite Owner,demog_ownrent_imp_Definite Renter,demog_ownrent_imp_Probable Owner,demog_ownrent_imp_Probable Renter,tier_1-Bronze,tier_2-Silver,tier_3-Gold,tier_4-Platinum,tier_5-Diamond,tier_prev_1-Bronze,tier_prev_2-Silver,tier_prev_3-Gold,tier_prev_4-Platinum,tier_prev_5-Diamond,tier_prev_xxx,P_churn_year_plus1_ind,P_log_margin_Y_pos_1,P_margin_Y_pos_1,CFV,retention_grp,margin_grp,quadrant_retention_margin
0,SC,29607,567,864,1,1,1,0,0,3,21,4.50,5,0,1,1-Bronze,xxx,I148760611,30NOV12,31MAY15,1,0.30,2.16,2.16,1.48,...,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,6.83e-01,4.7,85.94,33.99,1L,1L,1L_1L
1,MO,63901,632,573,1,1,1,0,0,9,9,36.00,1,1,0,1-Bronze,xxx,I151076142,29NOV13,29NOV14,1,0.30,1.32,1.32,1.32,...,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,8.57e-01,4.7,89.81,13.36,1L,1L,1L_1L
2,IL,60456,602,missing,0,1,1,0,0,1,167,3.07,17,0,0,1-Bronze,xxx,I27001315,05OCT00,NaN,0,0.70,2.78,3.46,1.56,...,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,6.22e-02,4.7,320.13,703.94,2H,2H,2H_2H
3,FL,34698,539,727,1,1,1,0,0,9,13,1.00,5,1,1,3-Gold,xxx,I150064127,31JUL13,29NOV14,1,0.60,2.62,2.62,1.93,...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,9.32e-01,4.7,313.88,20.65,1L,2H,1L_2H
4,CA,92084,825,760,0,1,1,0,0,1,2,1.00,2,0,0,1-Bronze,xxx,I152631565,30JUN14,NaN,0,0.48,2.08,2.08,1.79,...,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1.28e-01,4.7,171.19,328.52,1L,1L,1L_1L
5,CA,92084,825,760,0,1,1,0,0,1,2,1.00,2,0,0,1-Bronze,xxx,I152631565,30JUN14,NaN,0,0.48,2.08,2.08,1.79,...,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1.28e-01,4.7,171.19,328.52,1L,1L,1L_1L
6,MI,48446,505,810,1,0,1,0,0,11,128,1.58,4,0,3,1-Bronze,2-Silver,H62972005,04JAN04,30SEP14,1,0.78,2.49,3.30,1.89,...,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,7.50e-01,4.7,214.50,62.48,1L,2H,1L_2H
7,AL,35020,630,205,0,1,1,0,0,5,10,1.00,6,0,0,1-Bronze,xxx,I151044258,30OCT13,31MAR15,1,0.30,2.08,2.08,1.32,...,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,8.46e-01,4.7,171.19,27.86,1L,1L,1L_1L
8,AL,35020,630,205,0,1,1,0,0,5,10,1.00,6,0,0,1-Bronze,xxx,I151044258,30OCT13,31MAR15,1,0.30,2.08,2.08,1.32,...,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,8.46e-01,4.7,171.19,27.86,1L,1L,1L_1L
9,PR,missing,998,missing,0,1,1,0,1,1,121,2.07,30,0,0,1-Bronze,xxx,I95584028,04AUG04,NaN,0,0.48,2.78,3.08,1.32,...,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,8.23e-03,4.7,203.60,498.29,2H,2H,2H_2H


## Output scored data and go to Tableau for more analysis...

In [6]:
combined_df.to_csv(r'C:\Users\dangt\Desktop\Trinh Phan\OSU Lesson\Fall 2020\Customer Life Time Value\Group Project\CLV_scores.csv',index=False)

In [7]:
print("number of accounts, in each quadrant")
print(combined_df.groupby(['margin_grp', 'retention_grp'])['acct_ID'].count().apply("{:,}".format))

print("") # intentional blank line for spacing

print("percentage of accounts with scheduled purchase, in each quadrant")
print(combined_df.groupby(['margin_grp', 'retention_grp'])['scheduled_purchase_flg'].mean().apply("{:.1%}".format))

number of accounts, in each quadrant
margin_grp  retention_grp
1L          1L               106,828
            2H                33,399
2H          1L                48,754
            2H               122,182
Name: acct_ID, dtype: object

percentage of accounts with scheduled purchase, in each quadrant
margin_grp  retention_grp
1L          1L                0.4%
            2H               63.4%
2H          1L                0.2%
            2H               48.6%
Name: scheduled_purchase_flg, dtype: object


In [8]:
# using an agg function to summarize multiple metrics and create a dataframe for output to a .csv
# (this could be something you want to do for your team project work)

pivot_df = combined_df.groupby(['margin_grp', 'retention_grp']).agg({"acct_ID":"count", "mail_cnt_36mo":"mean", "purch_avg_interval":"median"})

# renaming columns in the pivot-table-like dataframe:
pivot_df = pivot_df.rename(columns={"acct_ID":"number of accounts", "mail_cnt_36mo":"average mail cnt", "purch_avg_interval":"purchase interval median"})

pivot_df.head(10)

number of accounts  average mail cnt  purchase interval median
margin_grp retention_grp                                                                
1L         1L                         106828              8.11                      2.27
           2H                          33399             12.86                      1.40
2H         1L                          48754             17.39                      1.29
           2H                         122182             31.01                      1.02

In [9]:
# saving this pivot-table summary as a .csv

pivot_df.to_csv(r'C:\Users\dangt\Desktop\Trinh Phan\OSU Lesson\Fall 2020\Customer Life Time Value\Group Project\pivot_table_1.csv', index=True)

In [ ]:
#